# 📊 NumPy Review for ML/DL

**Mục tiêu:** Ôn tập NumPy operations quan trọng cho ML/DL

**Nội dung:**
- Array creation & manipulation
- Indexing & slicing (fancy indexing)
- Broadcasting rules
- Linear algebra operations
- Random number generation
- Performance tips

**Level:** Intermediate (đã biết cơ bản)

---

In [ ]:
import numpy as np
import time

np.random.seed(42)
print(f"NumPy version: {np.__version__}")

---

## 1. Array Creation Patterns

### Common trong ML/DL

In [ ]:
# Zeros, ones (khởi tạo weights/biases)
weights = np.zeros((3, 4))
bias = np.ones(4)

# Identity (weight initialization)
identity = np.eye(5)

# Random (Xavier/He initialization pattern)
fan_in, fan_out = 128, 64
xavier = np.random.randn(fan_in, fan_out) * np.sqrt(2 / (fan_in + fan_out))
he = np.random.randn(fan_in, fan_out) * np.sqrt(2 / fan_in)

# Arange & linspace (learning rate schedules)
epochs = np.arange(0, 100)
lr_schedule = 0.1 * (0.95 ** epochs)  # Exponential decay

print(f"Xavier init shape: {xavier.shape}, mean: {xavier.mean():.4f}, std: {xavier.std():.4f}")
print(f"He init shape: {he.shape}, mean: {he.mean():.4f}, std: {he.std():.4f}")

## 2. Indexing & Slicing

### Basic vs Advanced Indexing

In [ ]:
# Sample data: batch of images (N, H, W, C)
images = np.random.rand(32, 28, 28, 3)

# Basic slicing (returns view)
first_10 = images[:10]  # First 10 images
red_channel = images[:, :, :, 0]  # Red channel only

# Boolean indexing (returns copy)
bright_pixels = images > 0.8
num_bright = bright_pixels.sum()

# Fancy indexing (returns copy)
indices = [0, 5, 10, 15]  # Select specific images
selected = images[indices]

# Advanced: Select pixels at specific coordinates
rows = [10, 15, 20]
cols = [5, 10, 15]
pixels = images[0, rows, cols, :]  # Shape: (3, 3)

print(f"Original shape: {images.shape}")
print(f"First 10 shape: {first_10.shape}")
print(f"Red channel shape: {red_channel.shape}")
print(f"Selected shape: {selected.shape}")
print(f"Bright pixels: {num_bright}")

### 💡 View vs Copy

- **View** (basic slicing): Shares memory, changes affect original
- **Copy** (boolean/fancy indexing): New memory, independent

```python
a = np.array([1, 2, 3])
b = a[:]          # View
c = a[[0, 1, 2]]  # Copy

b[0] = 999  # Changes a[0]
c[0] = 999  # Does NOT change a[0]
```

## 3. Broadcasting

### Rules & Common Patterns

In [ ]:
# Broadcasting rules:
# 1. Align shapes from right
# 2. Dimensions are compatible if: equal OR one is 1
# 3. Missing dimensions treated as 1

# Pattern 1: Add bias to each sample
X = np.random.randn(32, 10)  # (batch_size, features)
bias = np.random.randn(10)    # (features,)
result = X + bias             # Broadcasting: (32, 10) + (10,) -> (32, 10)

# Pattern 2: Normalize features (mean=0, std=1)
mean = X.mean(axis=0, keepdims=True)  # (1, 10)
std = X.std(axis=0, keepdims=True)    # (1, 10)
X_normalized = (X - mean) / std       # Broadcasting

# Pattern 3: Apply different scale per channel
images = np.random.rand(32, 28, 28, 3)  # (N, H, W, C)
channel_scale = np.array([0.5, 1.0, 1.5])  # (3,)
scaled = images * channel_scale  # Broadcasting: (32,28,28,3) * (3,) -> (32,28,28,3)

# Pattern 4: Distance matrix (all pairs)
points = np.random.randn(100, 2)  # 100 points, 2D
# Expand dims for broadcasting
p1 = points[:, np.newaxis, :]  # (100, 1, 2)
p2 = points[np.newaxis, :, :]  # (1, 100, 2)
distances = np.sqrt(((p1 - p2) ** 2).sum(axis=2))  # (100, 100)

print(f"X + bias: {result.shape}")
print(f"Normalized: mean={X_normalized.mean(axis=0)[:3]}, std={X_normalized.std(axis=0)[:3]}")
print(f"Distance matrix: {distances.shape}")

### ⚠️ Common Broadcasting Mistakes

```python
X = np.random.randn(32, 10)

# ❌ WRONG: Forgetting keepdims
mean = X.mean(axis=0)  # Shape: (10,)
X - mean  # Works, but conceptually unclear

# ✅ CORRECT: Use keepdims for clarity
mean = X.mean(axis=0, keepdims=True)  # Shape: (1, 10)
X - mean  # Clear broadcasting
```

## 4. Linear Algebra Operations

### Essential cho Neural Networks

In [ ]:
# Matrix multiplication
X = np.random.randn(32, 128)   # Input: (batch, in_features)
W = np.random.randn(128, 64)   # Weight: (in_features, out_features)
output = X @ W                 # or np.dot(X, W) -> (batch, out_features)

# Batch matrix multiplication (for attention, etc)
Q = np.random.randn(32, 8, 64)  # (batch, seq_len, d_k)
K = np.random.randn(32, 8, 64)  # (batch, seq_len, d_k)
attention = Q @ K.transpose(0, 2, 1)  # (32, 8, 8)
# Or use: np.einsum('bid,bjd->bij', Q, K)

# Frobenius norm (weight regularization)
l2_norm = np.linalg.norm(W)  # Default: Frobenius norm
l2_penalty = 0.01 * (l2_norm ** 2)

# Dot product (similarity)
v1 = np.random.randn(128)
v2 = np.random.randn(128)
similarity = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))  # Cosine similarity

print(f"Output shape: {output.shape}")
print(f"Attention shape: {attention.shape}")
print(f"L2 norm: {l2_norm:.4f}")
print(f"Cosine similarity: {similarity:.4f}")

### Einstein Summation (einsum) - Advanced

Powerful notation cho complex operations:

In [ ]:
# Example: Batch matrix multiplication
A = np.random.randn(32, 10, 20)  # (batch, i, j)
B = np.random.randn(32, 20, 15)  # (batch, j, k)

# Standard way
C1 = np.matmul(A, B)  # (32, 10, 15)

# Einsum way (more explicit)
C2 = np.einsum('bij,bjk->bik', A, B)  # Same result

# More examples
x = np.random.randn(32, 128)

# Batch dot product
diag = np.einsum('bi,bi->b', x, x)  # Sum of squares per sample

# Outer product
outer = np.einsum('i,j->ij', np.array([1,2,3]), np.array([4,5,6]))

print(f"Einsum result shape: {C2.shape}")
print(f"Results match: {np.allclose(C1, C2)}")
print(f"Batch dot product shape: {diag.shape}")

## 5. Axis Operations

### Reductions & Aggregations

In [ ]:
# Sample data: predictions and labels
logits = np.random.randn(32, 10)  # (batch_size, num_classes)
labels = np.random.randint(0, 10, size=32)  # True labels

# Softmax (numerical stable version)
def softmax(x, axis=-1):
    exp_x = np.exp(x - x.max(axis=axis, keepdims=True))  # Stability trick
    return exp_x / exp_x.sum(axis=axis, keepdims=True)

probs = softmax(logits, axis=1)  # (32, 10)

# Get predictions
preds = np.argmax(probs, axis=1)  # (32,)

# Accuracy
accuracy = (preds == labels).mean()

# Top-k accuracy
k = 3
top_k_preds = np.argsort(probs, axis=1)[:, -k:]  # Top-3 predictions
top_k_acc = np.any(top_k_preds == labels[:, np.newaxis], axis=1).mean()

# Per-class statistics
class_counts = np.bincount(labels, minlength=10)
class_correct = np.bincount(labels[preds == labels], minlength=10)
per_class_acc = class_correct / (class_counts + 1e-8)  # Avoid division by zero

print(f"Accuracy: {accuracy:.2%}")
print(f"Top-{k} Accuracy: {top_k_acc:.2%}")
print(f"Per-class accuracy (first 3): {per_class_acc[:3]}")
print(f"Probs sum to 1: {np.allclose(probs.sum(axis=1), 1.0)}")

### 💡 Axis Parameter Rules

```python
X = np.random.randn(32, 10, 20)  # (batch, time, features)

X.mean(axis=0)  # Average over batch -> (10, 20)
X.mean(axis=1)  # Average over time -> (32, 20)
X.mean(axis=2)  # Average over features -> (32, 10)
X.mean(axis=(0, 1))  # Average over batch and time -> (20,)
```

## 6. Random Number Generation

### Reproducibility & Modern API

In [ ]:
# OLD way (still works, but not recommended)
np.random.seed(42)
old_random = np.random.randn(5)

# NEW way (recommended for reproducibility)
rng = np.random.default_rng(42)  # Generator object
new_random = rng.standard_normal(5)

# Common distributions
uniform = rng.uniform(0, 1, size=(10, 10))  # Uniform [0, 1)
normal = rng.normal(0, 1, size=(10, 10))    # Normal N(0, 1)
integers = rng.integers(0, 10, size=100)    # Random integers [0, 10)

# Shuffle (data augmentation)
indices = np.arange(100)
rng.shuffle(indices)  # In-place shuffle

# Choice (sampling)
batch_indices = rng.choice(100, size=32, replace=False)  # Sample 32 without replacement

print(f"Old random: {old_random[:3]}")
print(f"New random: {new_random[:3]}")
print(f"Batch indices: {batch_indices[:5]}")

### 💡 Why Use Generator?

- **Thread-safe**: Multiple generators don't interfere
- **Reproducible**: Independent streams
- **Future-proof**: NumPy's recommended approach

```python
# Good practice for ML training
def train(seed=42):
    rng = np.random.default_rng(seed)
    # Use rng for all random operations
    X_shuffled = X[rng.permutation(len(X))]
```

## 7. Performance Tips

### Vectorization vs Loops

In [ ]:
# Task: Compute pairwise distances
points = np.random.randn(1000, 2)

# ❌ Slow: Python loops
def slow_distances(points):
    n = len(points)
    dist = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist[i, j] = np.sqrt(((points[i] - points[j]) ** 2).sum())
    return dist

# ✅ Fast: Vectorized
def fast_distances(points):
    # Broadcasting trick
    diff = points[:, np.newaxis, :] - points[np.newaxis, :, :]
    return np.sqrt((diff ** 2).sum(axis=2))

# Benchmark
small_points = np.random.randn(100, 2)

start = time.time()
d1 = slow_distances(small_points)
time_slow = time.time() - start

start = time.time()
d2 = fast_distances(small_points)
time_fast = time.time() - start

print(f"Slow method: {time_slow:.4f}s")
print(f"Fast method: {time_fast:.4f}s")
print(f"Speedup: {time_slow/time_fast:.1f}x")
print(f"Results match: {np.allclose(d1, d2)}")

### Memory Efficiency

```python
# ❌ Creates intermediate arrays
result = (X + Y) * Z

# ✅ In-place operations (careful!)
X += Y
X *= Z

# ✅ Use out parameter
result = np.empty_like(X)
np.add(X, Y, out=result)
np.multiply(result, Z, out=result)
```

## 8. Common Gotchas

### Avoid These Mistakes

In [ ]:
# Gotcha 1: Integer division
a = np.array([1, 2, 3])
print(f"a / 2 = {a / 2}")        # ✅ Float division
print(f"a // 2 = {a // 2}")      # Integer division

# Gotcha 2: Array modification during iteration
arr = np.array([1, 2, 3, 4, 5])
arr_copy = arr.copy()  # ✅ Make copy if needed

# Gotcha 3: Boolean array indexing
X = np.random.randn(10, 5)
mask = X > 0
positive = X[mask]  # Returns 1D array of all positive values (NOT shape of X)
print(f"Positive values shape: {positive.shape}")  # (N,) not (10, 5)

# Gotcha 4: NaN handling
arr_with_nan = np.array([1, 2, np.nan, 4, 5])
print(f"Mean (wrong): {arr_with_nan.mean()}")     # NaN
print(f"Mean (correct): {np.nanmean(arr_with_nan)}")  # ✅ Ignores NaN

# Gotcha 5: Floating point precision
print(f"0.1 + 0.2 == 0.3? {0.1 + 0.2 == 0.3}")  # False!
print(f"np.isclose(0.1 + 0.2, 0.3)? {np.isclose(0.1 + 0.2, 0.3)}")  # ✅ True

---

## 🎯 Key Takeaways

### Must Know for ML/DL

1. **Broadcasting**: Core của vectorization, hiểu shapes alignment
2. **Indexing**: Basic slicing (view) vs fancy indexing (copy)
3. **Axis operations**: Mean, sum, argmax với axis parameter
4. **Matrix operations**: @ operator, einsum cho complex ops
5. **Random generation**: Use `default_rng()` cho reproducibility
6. **Vectorization**: ALWAYS prefer vectorized operations over loops

### Common Patterns

```python
# Normalization
X_norm = (X - X.mean(axis=0, keepdims=True)) / X.std(axis=0, keepdims=True)

# Softmax
exp_x = np.exp(x - x.max(axis=-1, keepdims=True))
softmax = exp_x / exp_x.sum(axis=-1, keepdims=True)

# Batch operations
output = X @ W + b  # Broadcasting bias
```

### Performance Rules

1. Vectorize everything possible
2. Use views (slicing) when you can
3. Avoid Python loops on large arrays
4. Use appropriate dtypes (float32 vs float64)
5. Consider memory vs speed tradeoffs

---

**Next:** Pandas for data manipulation